# Example to understand the delete files and how they are used

# 1. Perform delete operation to generate delete files

### First we have to change dlete operation from CoW to MoR for the table on which delete opewration will be performed to ensure that delete files are written otherwise with CoW file will be rewritten without delete files.

In [5]:
%%sql

show create table hms.data_db.new_york_taxi;

createtab_stmt
"CREATE TABLE hms.data_db.new_york_taxi ( tpep_pickup_datetime TIMESTAMP, tpep_dropoff_datetime TIMESTAMP, passenger_count BIGINT, trip_distance DECIMAL(10,2), PULocationID BIGINT, DOLocationID BIGINT, fare_amount FLOAT)USING icebergPARTITIONED BY (days(tpep_pickup_datetime))LOCATION 's3a://lakehouse/warehouse/data_db/new_york_taxi'TBLPROPERTIES ( 'current-snapshot-id' = '3733467730074381361', 'format' = 'iceberg/parquet', 'format-version' = '2', 'sort-order' = 'PULocationID ASC NULLS FIRST, DOLocationID ASC NULLS FIRST', 'write.delete.mode' = 'merge-on-read', 'write.distribution-mode' = 'range', 'write.parquet.compression-codec' = 'zstd')"


In [4]:
%%sql

ALTER TABLE hms.data_db.new_york_taxi SET TBLPROPERTIES (
    'write.delete.mode'='merge-on-read'
);

++
||
++
++

### Confirm if the write mode has been changed or not

In [6]:
%%sql

show create table hms.data_db.new_york_taxi;

createtab_stmt
"CREATE TABLE hms.data_db.new_york_taxi ( tpep_pickup_datetime TIMESTAMP, tpep_dropoff_datetime TIMESTAMP, passenger_count BIGINT, trip_distance DECIMAL(10,2), PULocationID BIGINT, DOLocationID BIGINT, fare_amount FLOAT)USING icebergPARTITIONED BY (days(tpep_pickup_datetime))LOCATION 's3a://lakehouse/warehouse/data_db/new_york_taxi'TBLPROPERTIES ( 'current-snapshot-id' = '3733467730074381361', 'format' = 'iceberg/parquet', 'format-version' = '2', 'sort-order' = 'PULocationID ASC NULLS FIRST, DOLocationID ASC NULLS FIRST', 'write.delete.mode' = 'merge-on-read', 'write.distribution-mode' = 'range', 'write.parquet.compression-codec' = 'zstd')"


In [12]:
%%sql

-- Your SQL query here
--SELECT count(1) FROM hms.data_db.new_york_taxi WHERE DOLocationID = 234 LIMIT 10;
SELECT count(1) FROM hms.data_db.new_york_taxi WHERE DOLocationID = 234 and PULocationID = 231 LIMIT 10;

count(1)
508


In [3]:
%%sql

select * from hms.data_db.new_york_taxi.metadata_log_entries order by timestamp desc limit 5; 

timestamp,file,latest_snapshot_id,latest_schema_id,latest_sequence_number
2025-03-09 11:12:20.537000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00005-5710b475-60e8-4c47-ba13-24608460b4ea.metadata.json,3733467730074381361,0,3
2024-11-24 08:17:59.515000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00004-ef478216-63af-4e54-b497-342a08322aae.metadata.json,3733467730074381361,0,3
2024-11-22 05:11:42.701000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00003-e6da6690-093f-4026-9443-77cc4738d38e.metadata.json,6118885366905035378,0,2
2024-11-20 12:16:20.695000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00002-4aee5dc1-fdc6-413f-9363-b407e9ac41a7.metadata.json,7915680060403489000,0,1
2024-11-20 12:15:47.503000,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/00001-f26e1a62-3c31-4a77-82ef-bacf6ab54050.metadata.json,None,None,None


In [4]:
%%sql

delete from hms.data_db.new_york_taxi WHERE DOLocationID = 234 and PULocationID = 231;

25/03/10 05:36:02 WARN S3InputStream: An error occurred while aborting the stream
software.amazon.awssdk.core.exception.AbortedException
	at software.amazon.awssdk.core.exception.AbortedException$BuilderImpl.build(AbortedException.java:93)
	at software.amazon.awssdk.core.io.SdkFilterInputStream.abortIfNeeded(SdkFilterInputStream.java:45)
	at software.amazon.awssdk.core.io.SdkFilterInputStream.read(SdkFilterInputStream.java:59)
	at org.apache.iceberg.aws.s3.S3InputStream.abortStream(S3InputStream.java:218)
	at org.apache.iceberg.aws.s3.S3InputStream.closeStream(S3InputStream.java:202)
	at org.apache.iceberg.aws.s3.S3InputStream.close(S3InputStream.java:149)
	at org.apache.iceberg.avro.AvroIO$AvroInputStreamAdapter.close(AvroIO.java:82)
	at org.apache.iceberg.shaded.org.apache.avro.file.DataFileReader$SeekableInputStream.close(DataFileReader.java:325)
	at org.apache.iceberg.shaded.org.apache.avro.io.BinaryDecoder$InputStreamByteSource.close(BinaryDecoder.java:900)
	at org.apache.iceberg.

++
||
++
++

In [5]:
%%sql


select count(1) from hms.data_db.new_york_taxi WHERE DOLocationID = 234 and PULocationID = 231;

count(1)
0


In [7]:
%%sql

select * from hms.data_db.new_york_taxi.snapshots order by committed_at desc limit 2; 

committed_at,snapshot_id,parent_id,operation,manifest_list,summary
2025-03-10 05:36:06.148000,4990862803825292274,3733467730074381361,overwrite,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-4990862803825292274-1-ecc6807a-a9a4-4586-abb1-73d9dc7f0a5d.avro,"{'spark.app.id': 'local-1741584850436', 'changed-partition-count': '6', 'added-position-deletes': '508', 'total-equality-deletes': '0', 'total-position-deletes': '508', 'added-position-delete-files': '6', 'added-files-size': '10942', 'total-delete-files': '6', 'added-delete-files': '6', 'total-files-size': '5917742', 'total-records': '1048584', 'total-data-files': '18'}"
2024-11-24 08:17:59.515000,3733467730074381361,6118885366905035378,delete,s3a://lakehouse/warehouse/data_db/new_york_taxi/metadata/snap-3733467730074381361-1-abbed668-2863-4c9b-9278-897cc1b5d5b3.avro,"{'spark.app.id': 'local-1732436070168', 'removed-files-size': '2206', 'changed-partition-count': '1', 'total-equality-deletes': '0', 'deleted-data-files': '1', 'total-position-deletes': '0', 'total-delete-files': '0', 'deleted-records': '1', 'total-files-size': '5906800', 'total-records': '1048584', 'total-data-files': '18'}"


In [14]:
%%sql

select * from hms.data_db.new_york_taxi.files where content <> 0 limit 10; 

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
1,s3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2019-01-01/00000-5-a7e8951b-b721-49e5-9222-68a0e096fb3d-00001-deletes.parquet,PARQUET,0,"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 1))",63,1860,"{2147483546: 195, 2147483545: 189}",None,None,None,"{2147483546: bytearray(b's3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2019-01-01/00000-6-6d46122d-b309-45cb-b72c-8701bf47aa97-0-00009.parquet'), 2147483545: bytearray(b'\xc95\x02\x00\x00\x00\x00\x00')}","{2147483546: bytearray(b's3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2019-01-01/00000-6-6d46122d-b309-45cb-b72c-8701bf47aa97-0-00009.parquet'), 2147483545: bytearray(b'\x076\x02\x00\x00\x00\x00\x00')}",None,[4],None,None,"Row(DOLocationID=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), PULocationID=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), fare_amount=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), passenger_count=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), tpep_dropoff_datetime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), tpep_pickup_datetime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), trip_distance=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None))"
1,s3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2018-12-30/00000-5-a7e8951b-b721-49e5-9222-68a0e096fb3d-00002-deletes.parquet,PARQUET,0,"Row(tpep_pickup_datetime_day=datetime.date(2018, 12, 30))",1,1666,"{2147483546: 164, 2147483545: 40}",None,None,None,"{2147483546: bytearray(b's3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2018-12-30/00000-6-6d46122d-b309-45cb-b72c-8701bf47aa97-0-00007.parquet'), 2147483545: bytearray(b'\x07\x00\x00\x00\x00\x00\x00\x00')}","{2147483546: bytearray(b's3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2018-12-30/00000-6-6d46122d-b309-45cb-b72c-8701bf47aa97-0-00007.parquet'), 2147483545: bytearray(b'\x07\x00\x00\x00\x00\x00\x00\x00')}",None,[4],None,None,"Row(DOLocationID=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), PULocationID=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), fare_amount=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), passenger_count=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), tpep_dropoff_datetime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), tpep_pickup_datetime=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None), trip_distance=Row(column_size=None, value_count=None, null_value_count=None, nan_value_count=None, lower_bound=None, upper_bound=None))"
1,s3a://lakehouse/warehouse/data_db/new_york_taxi/data/tpep_pickup_datetime_day=2019-01-04/00000-5-a7e8951b-b721-49e5-9222-68a0e096fb3d-00003-deletes.parquet,PARQUET,0,"Row(tpep_pickup_datetime_day=datetime.date(2019, 1, 4))",124,1866,"{2147483546: 197, 21

In [15]:
%%sql

select * from hms.data_db.new_york_taxi.history; 

made_current_at,snapshot_id,parent_id,is_current_ancestor
2024-11-20 12:16:20.695000,7915680060403489000,None,True
2024-11-22 05:11:42.701000,6118885366905035378,7915680060403489000,True
2024-11-24 08:17:59.515000,3733467730074381361,6118885366905035378,True
2025-03-10 05:36:06.148000,4990862803825292274,3733467730074381361,True
